In [1]:
import pycosat
import pickle

In [2]:
constraintDict = pickle.load(open( "constraints.p", "rb" ))

In [3]:
class GridPuzzle:
    def __init__(self):
        #it is assumed that the number of variables in each dimension are equal
        self.dimensions = -1
        self.variableCount = 0
        self.dimensionDict = {}
        self.dimensionOrder = {}
        self.dimensionList = []
        self.variableOrder = {}
        self.generateFlag = False
        self.conditions = []
        self.parsedGridStatements = []
        self.parsedInputStatements = []
        self.varLookup = {}
        self.solutions = []
    
    def crashError(self, errorString, errorLocation):
        print(errorString)
        print('Error Location at '+errorLocation)
        print('Please Restart')
        
    def addDimension(self, dimName, variableList):
        
        if self.dimensions == -1:
            self.variableCount = len(variableList)
        else:
            if self.variableCount != len(variableList):
                self.crashError('Wrong number of variables!','addDimension')
        
        self.dimensionList.append(dimName)
        self.dimensionDict[dimName] = {}
        self.dimensionDict[dimName]['variables'] = variableList
        self.dimensions+=1
        self.dimensionOrder[self.dimensions] = dimName
        #self.dimensionOrder[self.dimensions]['Name'] = dimName
        

        for i,n in enumerate(variableList):
                self.dimensionDict[dimName][n] = i*(self.variableCount**self.dimensions)        
        
    def printState(self):
        print('Currently '+str(self.dimensions)+' in grid:')
        for k,v in self.dimensionDict.items():
            print('dimension '+k+' has')
            print(v)

    def addConditions(self, conditionStrings):
        validOps = ['<','>','=','!']
        for s in conditionStrings:
            a = s.split()
            if (len(a)==4 and a[1] not in validOps) or (len(a)==3 and a[1] not in validOps) :
                errorString='Condition Error: '+s+' contains invalid arguement'
                self.crashError(errorString,'addConditions')
            self.conditions.append(s)
        
    def generateParsedConditions(self):
        self.parsedGridStatements = constraintDict[self.dimensions+1][self.variableCount].copy()
        #print(self.parsedStatements)
        statements = self.parseConditions()
                
        self.parsedInputStatements = statements
        print(self.parsedInputStatements)
        
        
    def findSolutions(self):
        #len(list(pycosat.itersolve(l)))
        #for sol in pycosat.itersolve(l):
        #    print(sol)
        l = self.parsedGridStatements+self.parsedInputStatements
        #print(l)
        self.solutions = list(pycosat.itersolve(l))
        solCount = len(self.solutions)
        print('Found '+str(solCount)+' solutions')
        
    def parseConditions(self):
        conditionList = []
        #converts strings in specific formats into CNF logic statements that can be parsed by pycosat
        #each condition must have the form variable-sign-variable or variable-(<,>)-variable-dimension, like 'a = r' or 'a ! r' or 'a > b dim'
        #also, (a,b) and r should be variables from different dimensions
        for i in self.conditions:
            a = i.split()
            if len(a) == 4:
                #assume conditional is of the form 'a > b r' or 'a < b r'
                #where a and b are variables from a dimension, r is a variable in another dimension
                #The working assumption is that, when a>b, grid[a][r][...]>grid[b][r][...]
                #make sure to reference dimensions in the correct order!
                if a[2] == '>':
                    conditionList+self.genGreaterThan(a[0],a[2],a[3])
                elif i[2] == '<':
                    conditionList+self.genGreaterThan(a[2], a[0],a[3])
                else:
                    errorString='Condition Error: '+i+' contains invalid arguement'
                    self.crashError(errorString,'parseConditions1')
            elif len(a) == 3:
                if a[1] == '=':
                    #assume conditional is of the form 'a = r', where a and r are variables from different dimensions
                    #print(a[0],a[2])
                    conditionList.append(self.genEquals(a[0],a[2]))
                elif a[1] == '!':
                    #assume conditional of the form 'a ! r' where a and r are variables from different dimensions
                    conditionList+=self.genNotEquals(a[0],a[2])
                else:
                    errorString='Condition Error: '+i+' contains invalid arguement'
                    self.crashError(errorString,'parseConditions2')
            else:
                errorString='Condition Error: '+i+' contains invalid arguement'
                self.crashError(errorString,'parseConditions3')
        return conditionList

    def genEquals(self, a, r, var=0, level=0):
        #recursive function
        #loop through each dimension that a and r do not belong to
        #add variable at each loop to constraint list

        #enumerate dimensions 2, 3, and 4
        #a recursive algorithm would work also

        if level>self.dimensions:
            #if stack depth is greater than the number of dimensions, return var back up
            #print('Exit Condition!')
            #print('This var is '+str(var))
            return [var+1]

        conditionList = []
        dimension = self.dimensionOrder[level]
        #print(dimension)
        varList = self.dimensionDict[dimension]['variables']
        if a in varList:
            var+=self.dimensionDict[dimension][a]
            conditionList+=self.genEquals(a, r, var, level+1)
        elif r in varList:
            var+=self.dimensionDict[dimension][r]
            conditionList+=self.genEquals(a, r, var, level+1)
        else:
            #print('looping through variables in '+dimension)
            for i in varList:
                #print('checking variable '+i)
                varVal = self.dimensionDict[dimension][i]
                conditionList+=self.genEquals(a, r, var+varVal, level+1)
        return conditionList
                            
    def genNotEquals(self, a, r, var=0, level=0):
        #recursive function
        #loop through each dimension that a and r do not belong to
        #add variable at each loop to constraint list

        #enumerate dimensions 2, 3, and 4
        #a recursive algorithm would work also

        if level>self.dimensions:
            #if stack depth is greater than the number of dimensions, return var back up
            #print('Exit Condition!')
            #print('This var is '+str(var))
            return [[-(var+1)]]

        conditionList = []
        #print(var,level)
        dimension = self.dimensionOrder[level]
        varList = self.dimensionDict[dimension]['variables']
        #print(varList)
        if a in varList:
            #print('a matched!')
            var+=self.dimensionDict[dimension][a]
            #print('new var is '+str(var))
            conditionList+=self.genNotEquals(a, r, var, level+1)
        elif r in varList:
            var+=self.dimensionDict[dimension][r]
            conditionList+=self.genNotEquals(a, r, var, level+1)
        else:
            #print('looping through variables in '+dimension)
            for i in varList:
                #print('checking variable '+i)
                varVal = self.dimensionDict[dimension][i]
                conditionList+=self.genNotEquals(a, r, var+varVal, level+1)
        
        return conditionList
    
    def printSolution(self):
        #prints the first solution in the solutions list
        print(self.solutions[0])
        variableMatches = []
        for i in self.solutions[0]:
            if i > 0:
                variableMatches.append(self.decodeGridVariable(i))
        print(variableMatches)
        
    def decodeGridVariable(self, n):
        #determines the variables associated with a grid variable
        j = n-1
        variables = []
        for i in range(self.dimensions, -1, -1):
            varList = self.dimensionDict[self.dimensionOrder[i]]['variables']
            divisor = self.variableCount**i
            a = int(j/divisor)
            j = j%divisor
            variables.append(varList[a])
        return variables

In [4]:
grid=GridPuzzle()
grid.addDimension('Name',['John','Mary','Scott'])
grid.addDimension('Money',['1','2','3'])
grid.addDimension('Place',['T','S','B'])

In [5]:
grid.printState()

Currently 2 in grid:
dimension Name has
{'variables': ['John', 'Mary', 'Scott'], 'John': 0, 'Mary': 1, 'Scott': 2}
dimension Money has
{'variables': ['1', '2', '3'], '1': 0, '2': 3, '3': 6}
dimension Place has
{'variables': ['T', 'S', 'B'], 'T': 0, 'S': 9, 'B': 18}


In [6]:
grid.addConditions(['John = 1','Mary = 2','Scott = B','Mary ! T'])

In [7]:
grid.generateParsedConditions()

[[1, 10, 19], [5, 14, 23], [21, 24, 27], [-2], [-5], [-8]]


In [8]:
grid.findSolutions()

Found 1 solutions


In [9]:
grid.printSolution()

[1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, 14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, 27]
9
0 0
3
0 0
1
0 0
9
13 1
3
4 1
1
1 1
9
26 2
3
8 2
1
2 2
[['T', '1', 'John'], ['S', '2', 'Mary'], ['B', '3', 'Scott']]


In [ ]:
for i in range(4,1, -1):
    print(i)

In [ ]:
4%3